In [1]:
import sys
import warnings
import tensorflow as tf
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Dropout, Conv2D
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm

In [2]:
df = pd.read_csv('data/GOOG-year.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2016-11-02,778.200012,781.650024,763.450012,768.700012,768.700012,1872400
1,2016-11-03,767.250000,769.950012,759.030029,762.130005,762.130005,1943200
2,2016-11-04,750.659973,770.359985,750.560974,762.020020,762.020020,2134800
3,2016-11-07,774.500000,785.190002,772.549988,782.520020,782.520020,1585100
4,2016-11-08,783.400024,795.632996,780.190002,790.510010,790.510010,1350800


In [3]:
minmax = MinMaxScaler().fit(df.iloc[:, 4:5].astype('float32'))
df_log = minmax.transform(df.iloc[:, 4:5].astype('float32')) 
df_log = pd.DataFrame(df_log, columns=['Close'])
df_log.head()

,Close
0,0.112708
1,0.090008
2,0.089628
3,0.160459
4,0.188066


In [4]:
test_size = 30
simulation_size = 10

df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]
df.shape, df_train.shape, df_test.shape

((252, 7), (222, 1), (30, 1))

In [5]:
# 
X, Y = [], []
for k in range(0, df_train.shape[0] - 1, 30):
    index = min(k + 30, df_train.shape[0] - 1)
    x = df_train.iloc[k : index, :].values
    y = df_train.iloc[k + 1 : index + 1, :].values
    X.append(x)
    Y.append(y)

In [6]:
num_layers = 1
size_layer = 128
timestamp = test_size
epoch = 300
dropout_rate = 0.7
future_day = test_size
learning_rate = 1e-3

In [14]:
# Model

def encoder_block(n_hidden, filter_size):
    def f(input):
        out = tf.expand_dims(input, 2)
        out = Conv2D(filters=n_hidden, kernel_size=filter_size, padding='same', activation=None)(out)
        out = tf.squeeze(out, 2)
        return out
    return f

def decoder_block(n_hidden, filter_size):
    def f(input):
        out = tf.expand_dims(input, 2)
        out = Conv2D(filters=n_hidden, kernel_size=filter_size, padding='same', activation=None)(out)
        out = tf.squeeze(out, 2)
        return out
    return f


def glu(x):
    return tf.multiply(
        x[:, :, : tf.shape(x)[2] // 2],
        tf.sigmoid(x[:, :, tf.shape(x)[2] // 2 :]),
    )

def layer(conv_block, kernel_size, n_hidden, residual):
    def f(input, residual=residual):
        z = conv_block(n_hidden, (kernel_size, 1))(input)
        return glu(z) + (residual if residual is not None else 0)
    return f

def cnn_seq2seq_model(size_layer=128, kernel_size=3, n_block=1, n_attn_heads=16, output_size=1, traing=True):
    
    inputs = Input(shape=(30,1))
    # FC
    encoder_embedded  = Dense(128)(inputs)
    e = tf.identity(encoder_embedded )
    # Encoder
    for i in range(n_block):
        z = layer(encoder_block, kernel_size, size_layer * 2, encoder_embedded)(encoder_embedded)
        if traing:
            z = Dropout(0.7)(z)
        encoder_embedded = z
        
    encoder_output, output_memory = encoder_embedded , encoder_embedded  + e
    g = tf.identity(encoder_embedded)                 
    # Decoder
    for i in range(n_block):
        attn_res = h = layer(decoder_block, kernel_size, size_layer * 2, residual = tf.zeros_like(encoder_embedded))(encoder_embedded)
        C = []
        for j in range(n_attn_heads):
            h_ = Dense(size_layer // n_attn_heads)(h)
            g_ = Dense(size_layer // n_attn_heads)(g)
            zu_ = Dense(size_layer // n_attn_heads)(encoder_output)
            ze_ = Dense(size_layer // n_attn_heads)(output_memory)

            d = Dense(size_layer // n_attn_heads)(h_) + g_
            dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
            a = tf.nn.softmax(dz)
            c_ = tf.matmul(a, ze_)
            C.append(c_)

    c = tf.concat(C, 2)
    h = Dense(size_layer)(attn_res + c)
    h = Dropout(0.7)(h)
    encoder_embedded = h

    encoder_embedded = tf.sigmoid(encoder_embedded[-1])
    outputs = Dense(output_size)(encoder_embedded)           
    model = Model(inputs=inputs, outputs=outputs)
    return model

In [15]:
model = cnn_seq2seq_model()
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 30, 1)]      0                                            
__________________________________________________________________________________________________
dense_164 (Dense)               (None, 30, 128)      256         input_3[0][0]                    
__________________________________________________________________________________________________
tf.expand_dims_4 (TFOpLambda)   (None, 30, 1, 128)   0           dense_164[0][0]                  
__________________________________________________________________________________________________
conv2d_4 (Conv2D)               (None, 30, 1, 256)   98560       tf.expand_dims_4[0][0]           
____________________________________________________________________________________________

In [9]:
tf.keras.utils.plot_model(model, to_file='model.png')

('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


ModuleNotFoundError: No module named 'pydotplus'